<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>


# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install transformers
!pip install torch

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib as mpl
import torch
import transformers as ppb # pytorch transformers
from tqdm import notebook
import warnings

import scipy.stats as st
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import (r2_score,
                             mean_absolute_error,
                             mean_squared_error,
                             precision_score,
                             recall_score,
                             accuracy_score,
                             f1_score,
                             fbeta_score,
                             roc_curve,
                             roc_auc_score,
                             RocCurveDisplay,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             make_scorer
                            )


from sklearn.model_selection import (train_test_split,
                                     GridSearchCV,
                                     RandomizedSearchCV,
                                     cross_val_score
                                    )

from sklearn.preprocessing import (StandardScaler,
                                   MinMaxScaler,
                                   RobustScaler,
                                   OneHotEncoder,
                                   LabelEncoder,
                                   OrdinalEncoder
                                  )


RANDOM_STATE = 42
TEST_SIZE = 0.25

warnings.filterwarnings('ignore')

sb.set(style = 'whitegrid')
sb.set(rc={'figure.figsize':(16,9)})
sb.set_context('paper', rc={'font.size': 10,
                             'axes.titlesize': 10,
                             'xtrick.labelsize': 'small',
                             'ytrick.labelsize': 'small',
                             'legend.fontsize': 'small',
                             'legend.title_fontsize':10})

In [3]:
#data = pd.read_csv('C:/Users/Ilya/Downloads/toxic_comments.csv', header=None)


try:

    data = pd.read_csv('C:/Users/Ilya/Downloads/toxic_comments.csv', header=None)

except:

    data = pd.read_csv('/datasets/toxic_comments.csv', header=None)

In [4]:
display(data.head())
data.info()

,0,1,2
0,NaN,text,toxic
1,0.0,Explanation\nWhy the edits made under my usern...,0
2,1.0,D'aww! He matches this background colour I'm s...,0
3,2.0,"Hey man, I'm really not trying to edit war. It...",0
4,3.0,"""\nMore\nI can't make any real suggestions on ...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159293 entries, 0 to 159292
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       159292 non-null  float64
 1   1       159293 non-null  object 
 2   2       159293 non-null  object 
dtypes: float64(1), object(2)
memory usage: 3.6+ MB


In [5]:
data = data.drop(0,axis=1)
data = data.drop(0,axis=0)

data_droped = data.copy()
data_droped = data_droped[data_droped[1].str.len() <= 500] 

batch_1 = data_droped[:2000]

In [6]:
batch_1[2].value_counts()

2
0    1771
1     229
Name: count, dtype: int64

In [7]:
features_train, features_test, labels_train, labels_test = train_test_split(
    data_droped[1],
    data_droped[2], 
    train_size=0.98, 
    random_state=42,
    stratify=data_droped[2])

print(f"Количество строк в labels_train по классам: {np.bincount(labels_train)}")
print(f"Количество строк в labels_test по классам: {np.bincount(labels_test)}")

Количество строк в labels_train по классам: [109098  13834]
Количество строк в labels_test по классам: [2227  282]


In [9]:
# pd.set_option('display.max_colwidth', None)  # Отключаем ограничение ширины колонки
# batch_1.head(15)

## Обучение

### Обучение BERT

In [10]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)

model = ppb.AutoModel.from_pretrained('unitary/toxic-bert')
tokenizer = ppb.AutoTokenizer.from_pretrained('unitary/toxic-bert')

In [11]:
tokenized = features_test.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [12]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [13]:
np.array(padded).shape

(2509, 166)

In [14]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2509, 166)

In [15]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [16]:
features = last_hidden_states[0][:,0,:].numpy()

In [17]:
labels = labels_test

In [18]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [19]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [20]:
#lr_clf.f1_score(test_features, test_labels)
pred_labels=pd.Series(lr_clf.predict(test_features))
test_labels = [int(label) for label in test_labels]
pred_labels = [int(label) for label in pred_labels]

In [21]:
print('f1_score: %.2f' % f1_score(test_labels, pred_labels))  #,average='weighted'

f1_score: 0.95


### Обучение с использованием эмбандингов

In [22]:
batch_size = 100
embeddings = []
padded_2 = padded

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    # Создание батчей
    batch = torch.tensor(padded[batch_size * i:batch_size * (i + 1)], dtype=torch.long)
    attention_mask_batch = torch.tensor(attention_mask[batch_size * i:batch_size * (i + 1)], dtype=torch.long)

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:, 0, :].numpy())

  0%|          | 0/25 [00:00<?, ?it/s]

In [23]:
features = np.concatenate(embeddings)

labels_test_1 = labels_test[:2500]

X = features
y = labels_test_1

X_train, X_test, y_train, y_test = train_test_split(
 X, y, test_size=0.5, random_state=RANDOM_STATE
)
 

lr_clf_1 = LogisticRegression()
lr_clf_1.fit(X_train, y_train)
    
 #lr_clf.f1_score(test_features, test_labels)
y_pred=pd.Series(lr_clf.predict(X_test))
y_test = [int(label) for label in y_test]
y_pred = [int(label) for label in y_pred]   

print('f1_score: %.2f' % f1_score(y_test, y_pred))

f1_score: 0.98


## Выводы

Для выполнения задания была обучена модель логистической регрессии, показатель f1 - 0.8 , что соответствует техническому заданию. 
